In [1]:
import numpy as np
import math
import roboticstoolbox as rtb
from spatialmath.base import *
from spatialmath import SE3
import spatialmath.base.symbolic as sym



In [2]:
theta = sym.symbol('theta')
theta

theta

In [3]:
type(theta)

sympy.core.symbol.Symbol

In [4]:
theta.is_real

True

In [6]:
R = rotx(theta)

In [7]:
R

array([[1, 0, 0],
       [0, cos(theta), -sin(theta)],
       [0, sin(theta), cos(theta)]], dtype=object)

In [8]:
T = trotx(theta)

In [9]:
T

array([[1, 0, 0, 0],
       [0, cos(theta), -sin(theta), 0],
       [0, sin(theta), cos(theta), 0],
       [0, 0, 0, 1]], dtype=object)

In [11]:
type(T[1,1])

cos

In [12]:
T@T

array([[1, 0, 0, 0],
       [0, -sin(theta)**2 + cos(theta)**2, -2*sin(theta)*cos(theta), 0],
       [0, 2*sin(theta)*cos(theta), -sin(theta)**2 + cos(theta)**2, 0],
       [0, 0, 0, 1]], dtype=object)

In [13]:
T = SE3.Rx(theta)

In [14]:
T

   1         0         0         0         
   0        cos(theta)   -sin(theta)   0         
   0        sin(theta)   cos(theta)    0         
   0         0         0         1         


In [16]:
T2 = T*T

In [17]:
T2

   1        0            0             0         
  0            -sin(theta)**2 + cos(theta)**2 -2*sin(theta)*cos(theta) 0             
  0            2*sin(theta)*cos(theta) -sin(theta)**2 + cos(theta)**2 0             
   0        0            0             1         


In [18]:
puma = rtb.models.DH.Puma560(symbolic=True)

In [19]:
print(puma)

DHRobot: Puma 560 (by Unimation), 6 joints (RRRRRR), dynamics, geometry, standard DH parameters
┌────┬────────┬────────┬───────┬─────────┬────────┐
│θⱼ  │   dⱼ   │   aⱼ   │  ⍺ⱼ   │   q⁻    │   q⁺   │
├────┼────────┼────────┼───────┼─────────┼────────┤
│ q1 │ 0.6718 │      0 │  pi/2 │ -160.0° │ 160.0° │
│ q2 │      0 │ 0.4318 │     0 │ -110.0° │ 110.0° │
│ q3 │   0.15 │ 0.0203 │ -pi/2 │ -135.0° │ 135.0° │
│ q4 │ 0.4318 │      0 │  pi/2 │ -266.0° │ 266.0° │
│ q5 │      0 │      0 │ -pi/2 │ -100.0° │ 100.0° │
│ q6 │      0 │      0 │     0 │ -266.0° │ 266.0° │
└────┴────────┴────────┴───────┴─────────┴────────┘

┌─┬──┐
└─┴──┘

┌─────┬─────┬──────────────────────┬───────────────────────┬─────┬──────────────────────┬─────┐
│name │ q0  │ q1                   │ q2                    │ q3  │ q4                   │ q5  │
├─────┼─────┼──────────────────────┼───────────────────────┼─────┼──────────────────────┼─────┤
│  qr │  0° │ 28.6478897565412*pi° │ -28.6478897565412*pi° │  0° │  0°          

In [20]:
q = sym.symbol('q_:6')

In [21]:
q

(q_0, q_1, q_2, q_3, q_4, q_5)

In [22]:
q[0]

q_0

In [23]:
T = puma.fkine(q)

In [24]:
T

  (((-sin(q_1)*sin(q_2)*cos(q_0) + cos(q_0)*cos(q_1)*cos(q_2))*cos(q_3) - sin(q_0)*sin(q_3))*cos(q_4) + (-sin(q_1)*cos(q_0)*cos(q_2) - sin(q_2)*cos(q_0)*cos(q_1))*sin(q_4))*cos(q_5) + (-(-sin(q_1)*sin(q_2)*cos(q_0) + cos(q_0)*cos(q_1)*cos(q_2))*sin(q_3) - sin(q_0)*cos(q_3))*sin(q_5) -(((-sin(q_1)*sin(q_2)*cos(q_0) + cos(q_0)*cos(q_1)*cos(q_2))*cos(q_3) - sin(q_0)*sin(q_3))*cos(q_4) + (-sin(q_1)*cos(q_0)*cos(q_2) - sin(q_2)*cos(q_0)*cos(q_1))*sin(q_4))*sin(q_5) + (-(-sin(q_1)*sin(q_2)*cos(q_0) + cos(q_0)*cos(q_1)*cos(q_2))*sin(q_3) - sin(q_0)*cos(q_3))*cos(q_5) -((-sin(q_1)*sin(q_2)*cos(q_0) + cos(q_0)*cos(q_1)*cos(q_2))*cos(q_3) - sin(q_0)*sin(q_3))*sin(q_4) + (-sin(q_1)*cos(q_0)*cos(q_2) - sin(q_2)*cos(q_0)*cos(q_1))*cos(q_4) 0.15005*sin(q_0) - 0.0203*sin(q_1)*sin(q_2)*cos(q_0) - 0.4318*sin(q_1)*cos(q_0)*cos(q_2) - 0.4318*sin(q_2)*cos(q_0)*cos(q_1) + 0.0203*cos(q_0)*cos(q_1)*cos(q_2) + 0.4318*cos(q_0)*cos(q_1)  
  (((-sin(q_0)*sin(q_1)*sin(q_2) + sin(q_0)*cos(q_1)*cos(q_2))*cos(q_3) +

In [25]:
T[0]

  (((-sin(q_1)*sin(q_2)*cos(q_0) + cos(q_0)*cos(q_1)*cos(q_2))*cos(q_3) - sin(q_0)*sin(q_3))*cos(q_4) + (-sin(q_1)*cos(q_0)*cos(q_2) - sin(q_2)*cos(q_0)*cos(q_1))*sin(q_4))*cos(q_5) + (-(-sin(q_1)*sin(q_2)*cos(q_0) + cos(q_0)*cos(q_1)*cos(q_2))*sin(q_3) - sin(q_0)*cos(q_3))*sin(q_5) -(((-sin(q_1)*sin(q_2)*cos(q_0) + cos(q_0)*cos(q_1)*cos(q_2))*cos(q_3) - sin(q_0)*sin(q_3))*cos(q_4) + (-sin(q_1)*cos(q_0)*cos(q_2) - sin(q_2)*cos(q_0)*cos(q_1))*sin(q_4))*sin(q_5) + (-(-sin(q_1)*sin(q_2)*cos(q_0) + cos(q_0)*cos(q_1)*cos(q_2))*sin(q_3) - sin(q_0)*cos(q_3))*cos(q_5) -((-sin(q_1)*sin(q_2)*cos(q_0) + cos(q_0)*cos(q_1)*cos(q_2))*cos(q_3) - sin(q_0)*sin(q_3))*sin(q_4) + (-sin(q_1)*cos(q_0)*cos(q_2) - sin(q_2)*cos(q_0)*cos(q_1))*cos(q_4) 0.15005*sin(q_0) - 0.0203*sin(q_1)*sin(q_2)*cos(q_0) - 0.4318*sin(q_1)*cos(q_0)*cos(q_2) - 0.4318*sin(q_2)*cos(q_0)*cos(q_1) + 0.0203*cos(q_0)*cos(q_1)*cos(q_2) + 0.4318*cos(q_0)*cos(q_1)  
  (((-sin(q_0)*sin(q_1)*sin(q_2) + sin(q_0)*cos(q_1)*cos(q_2))*cos(q_3) +

In [26]:
from sympy import Matrix
Matrix(T.A)

Matrix([
[(((-sin(q_1)*sin(q_2)*cos(q_0) + cos(q_0)*cos(q_1)*cos(q_2))*cos(q_3) - sin(q_0)*sin(q_3))*cos(q_4) + (-sin(q_1)*cos(q_0)*cos(q_2) - sin(q_2)*cos(q_0)*cos(q_1))*sin(q_4))*cos(q_5) + (-(-sin(q_1)*sin(q_2)*cos(q_0) + cos(q_0)*cos(q_1)*cos(q_2))*sin(q_3) - sin(q_0)*cos(q_3))*sin(q_5), -(((-sin(q_1)*sin(q_2)*cos(q_0) + cos(q_0)*cos(q_1)*cos(q_2))*cos(q_3) - sin(q_0)*sin(q_3))*cos(q_4) + (-sin(q_1)*cos(q_0)*cos(q_2) - sin(q_2)*cos(q_0)*cos(q_1))*sin(q_4))*sin(q_5) + (-(-sin(q_1)*sin(q_2)*cos(q_0) + cos(q_0)*cos(q_1)*cos(q_2))*sin(q_3) - sin(q_0)*cos(q_3))*cos(q_5), -((-sin(q_1)*sin(q_2)*cos(q_0) + cos(q_0)*cos(q_1)*cos(q_2))*cos(q_3) - sin(q_0)*sin(q_3))*sin(q_4) + (-sin(q_1)*cos(q_0)*cos(q_2) - sin(q_2)*cos(q_0)*cos(q_1))*cos(q_4),  0.15005*sin(q_0) - 0.0203*sin(q_1)*sin(q_2)*cos(q_0) - 0.4318*sin(q_1)*cos(q_0)*cos(q_2) - 0.4318*sin(q_2)*cos(q_0)*cos(q_1) + 0.0203*cos(q_0)*cos(q_1)*cos(q_2) + 0.4318*cos(q_0)*cos(q_1)],
[(((-sin(q_0)*sin(q_1)*sin(q_2) + sin(q_0)*cos(q_1)*cos(q_2))

In [27]:
Ts = T.simplify()

In [28]:
M = Matrix(Ts.A)
M

Matrix([
[-((sin(q_0)*sin(q_3) - cos(q_0)*cos(q_3)*cos(q_1 + q_2))*cos(q_4) + sin(q_4)*sin(q_1 + q_2)*cos(q_0))*cos(q_5) - (sin(q_0)*cos(q_3) + sin(q_3)*cos(q_0)*cos(q_1 + q_2))*sin(q_5),   ((sin(q_0)*sin(q_3) - cos(q_0)*cos(q_3)*cos(q_1 + q_2))*cos(q_4) + sin(q_4)*sin(q_1 + q_2)*cos(q_0))*sin(q_5) - (sin(q_0)*cos(q_3) + sin(q_3)*cos(q_0)*cos(q_1 + q_2))*cos(q_5),  (sin(q_0)*sin(q_3) - cos(q_0)*cos(q_3)*cos(q_1 + q_2))*sin(q_4) - sin(q_1 + q_2)*cos(q_0)*cos(q_4),  0.15005*sin(q_0) - 0.4318*sin(q_1 + q_2)*cos(q_0) + 0.4318*cos(q_0)*cos(q_1) + 0.0203*cos(q_0)*cos(q_1 + q_2)],
[ ((sin(q_0)*cos(q_3)*cos(q_1 + q_2) + sin(q_3)*cos(q_0))*cos(q_4) - sin(q_0)*sin(q_4)*sin(q_1 + q_2))*cos(q_5) - (sin(q_0)*sin(q_3)*cos(q_1 + q_2) - cos(q_0)*cos(q_3))*sin(q_5), (-(sin(q_0)*cos(q_3)*cos(q_1 + q_2) + sin(q_3)*cos(q_0))*cos(q_4) + sin(q_0)*sin(q_4)*sin(q_1 + q_2))*sin(q_5) + (-sin(q_0)*sin(q_3)*cos(q_1 + q_2) + cos(q_0)*cos(q_3))*cos(q_5), -(sin(q_0)*cos(q_3)*cos(q_1 + q_2) + sin(q_3)*cos(q_0))*sin(q

In [29]:
M[:3,3]

Matrix([
[ 0.15005*sin(q_0) - 0.4318*sin(q_1 + q_2)*cos(q_0) + 0.4318*cos(q_0)*cos(q_1) + 0.0203*cos(q_0)*cos(q_1 + q_2)],
[-0.4318*sin(q_0)*sin(q_1 + q_2) + 0.4318*sin(q_0)*cos(q_1) + 0.0203*sin(q_0)*cos(q_1 + q_2) - 0.15005*cos(q_0)],
[                                     0.4318*sin(q_1) + 0.0203*sin(q_1 + q_2) + 0.4318*cos(q_1 + q_2) + 0.67183]])

In [30]:
from sympy import ccode, pycode, octave_code
print(ccode(M, assign_to="T"))

T[0] = -((sin(q_0)*sin(q_3) - cos(q_0)*cos(q_3)*cos(q_1 + q_2))*cos(q_4) + sin(q_4)*sin(q_1 + q_2)*cos(q_0))*cos(q_5) - (sin(q_0)*cos(q_3) + sin(q_3)*cos(q_0)*cos(q_1 + q_2))*sin(q_5);
T[1] = ((sin(q_0)*sin(q_3) - cos(q_0)*cos(q_3)*cos(q_1 + q_2))*cos(q_4) + sin(q_4)*sin(q_1 + q_2)*cos(q_0))*sin(q_5) - (sin(q_0)*cos(q_3) + sin(q_3)*cos(q_0)*cos(q_1 + q_2))*cos(q_5);
T[2] = (sin(q_0)*sin(q_3) - cos(q_0)*cos(q_3)*cos(q_1 + q_2))*sin(q_4) - sin(q_1 + q_2)*cos(q_0)*cos(q_4);
T[3] = 0.15004999999999999*sin(q_0) - 0.43180000000000002*sin(q_1 + q_2)*cos(q_0) + 0.43180000000000002*cos(q_0)*cos(q_1) + 0.020299999999999999*cos(q_0)*cos(q_1 + q_2);
T[4] = ((sin(q_0)*cos(q_3)*cos(q_1 + q_2) + sin(q_3)*cos(q_0))*cos(q_4) - sin(q_0)*sin(q_4)*sin(q_1 + q_2))*cos(q_5) - (sin(q_0)*sin(q_3)*cos(q_1 + q_2) - cos(q_0)*cos(q_3))*sin(q_5);
T[5] = (-(sin(q_0)*cos(q_3)*cos(q_1 + q_2) + sin(q_3)*cos(q_0))*cos(q_4) + sin(q_0)*sin(q_4)*sin(q_1 + q_2))*sin(q_5) + (-sin(q_0)*sin(q_3)*cos(q_1 + q_2) + cos(q_0)*cos(

In [31]:
print(pycode(M))

ImmutableDenseMatrix([[-((math.sin(q_0)*math.sin(q_3) - math.cos(q_0)*math.cos(q_3)*math.cos(q_1 + q_2))*math.cos(q_4) + math.sin(q_4)*math.sin(q_1 + q_2)*math.cos(q_0))*math.cos(q_5) - (math.sin(q_0)*math.cos(q_3) + math.sin(q_3)*math.cos(q_0)*math.cos(q_1 + q_2))*math.sin(q_5), ((math.sin(q_0)*math.sin(q_3) - math.cos(q_0)*math.cos(q_3)*math.cos(q_1 + q_2))*math.cos(q_4) + math.sin(q_4)*math.sin(q_1 + q_2)*math.cos(q_0))*math.sin(q_5) - (math.sin(q_0)*math.cos(q_3) + math.sin(q_3)*math.cos(q_0)*math.cos(q_1 + q_2))*math.cos(q_5), (math.sin(q_0)*math.sin(q_3) - math.cos(q_0)*math.cos(q_3)*math.cos(q_1 + q_2))*math.sin(q_4) - math.sin(q_1 + q_2)*math.cos(q_0)*math.cos(q_4), 0.15005*math.sin(q_0) - 0.4318*math.sin(q_1 + q_2)*math.cos(q_0) + 0.4318*math.cos(q_0)*math.cos(q_1) + 0.0203*math.cos(q_0)*math.cos(q_1 + q_2)], [((math.sin(q_0)*math.cos(q_3)*math.cos(q_1 + q_2) + math.sin(q_3)*math.cos(q_0))*math.cos(q_4) - math.sin(q_0)*math.sin(q_4)*math.sin(q_1 + q_2))*math.cos(q_5) - (math.s

In [32]:
from sympy import lambdify
T_func = lambdify(q, M, modules='numpy')

In [33]:
T_func(0, 0, 0, 0, 0, 0)

array([[ 1.     , -0.     , -0.     ,  0.4521 ],
       [ 0.     ,  1.     , -0.     , -0.15005],
       [ 0.     , -0.     ,  1.     ,  1.10363],
       [ 0.     ,  0.     ,  0.     ,  1.     ]])

In [34]:
g = sym.symbol('g')
puma.gravity = [0, 0, g]

In [35]:
qd = sym.symbol('qd_:6')
qd

(qd_0, qd_1, qd_2, qd_3, qd_4, qd_5)

In [36]:
qdd = sym.symbol('qdd_:6')
qdd

(qdd_0, qdd_1, qdd_2, qdd_3, qdd_4, qdd_5)

In [37]:
%time tau = puma.rne_python(q, qd, qdd)

Wall time: 325 ms


In [38]:
from sympy import trigsimp, simplify